5. Aplicar el Algoritmo REV2 al siguiente set de datos de reviews de productos de Amazon, para detectar potenciales usuarios maliciosos y otros ciertamente honestos. Por simplificación (y unificación de criterios), considerar $γ_1 = γ_2 = 0.5$. Obtener aquellos usuarios cuya justicia (fairness) es menor o igual a 0.2 (son maliciosos) y tienen al menos 5 reviews, así como la proporción de nodos que son extremadamente justos: aquellos con justicia mayor o igual a 0.9, y con al
menos 10 reviews (aristas de salida). 

**[2 Puntos]**

In [1]:
import pandas as pd
import numpy as np

In [2]:
reviews_df = pd.read_csv('../data/ratings_Electronics.csv')

In [3]:
reviews_df.head()

,User ID,Product ID,Rating,Time Stamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [4]:
reviews_df["Rating"] = np.where(reviews_df["Rating"] < 2.5, -1, 1)

In [5]:
reviews_df.head()

,User ID,Product ID,Rating,Time Stamp
0,AKM1MP6P0OYPR,0132793040,1,1365811200
1,A2CX7LUOHB2NDG,0321732944,1,1341100800
2,A2NWSAGRHCP8N5,0439886341,-1,1367193600
3,A2WNBOD3WNDNKT,0439886341,1,1374451200
4,A1GI0U4ZRJA8WN,0439886341,-1,1334707200


In [6]:
import networkx as nx

In [7]:
reviews_nx = nx.from_pandas_edgelist(reviews_df, source='User ID', target='Product ID', edge_attr='Rating',create_using=nx.DiGraph())

In [8]:
fairness = {}
reliability = {}
goodness = {}

In [9]:
y1 = 0.5
y2 = 0.5

In [10]:
def reliability_score(y1,y2,fairness,score,goodness):
    return (1/(y1+y2)) * (y1*fairness + y2*(1-((abs(score-goodness))/2)))

In [11]:
for i in range(10):
    # actualizar el fairness 
    for user in reviews_nx:
        if reviews_nx.out_degree(user) > 1: # si no tiene salidas es un producto
            reliability_sum = 0
            for product in reviews_nx[user].keys():
                reliability_sum += reliability.get((user,product),1)
            fairness[user] = reliability_sum / reviews_nx.out_degree(user)

    # actualizar el goodness
    for product in reviews_nx:
        if reviews_nx.out_degree(product) == 0: # si no tiene salidas es un producto
            reliability_x_score_sum = 0
            for user,_ in reviews_nx.in_edges(product):
                reliability_x_score_sum += reliability.get((user,product),1)*reviews_nx[user][product]["Rating"]
            goodness[product] = reliability_x_score_sum/reviews_nx.in_degree(product)

    # actualizar el reliability
    for user in reviews_nx:
        if reviews_nx.out_degree(user) > 1: # si no tiene salidas es un producto
            for product in reviews_nx[user].keys():
                reliability[(user,product)] = reliability_score(y1,y2,fairness[user],reviews_nx[user][product]["Rating"],goodness[product])
    print("Iteración: ", i)

Iteración:  0
Iteración:  1
Iteración:  2
Iteración:  3


KeyboardInterrupt: 

In [ ]:
usuarios_maliciosos = {}
usuarios_justos = {}
for user in reviews_nx:
    if reviews_nx.out_degree(user) > 1: # si no tiene salidas es un producto
        if fairness[user] < 0.2 and reviews_nx.out_degree(user) > 5:
            usuarios_maliciosos[user] = fairness[user]
        elif fairness[user] > 0.9 and reviews_nx.out_degree(user) > 10:
            usuarios_justos[user] = fairness[user]          

: 